In [1]:
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification

In [2]:
data = pd.read_csv('../Anomaly-Detection/amazon_reviews.txt', delimiter="\t")
features = ['RATING', 'REVIEW_TEXT', 'VERIFIED_PURCHASE', 'LABEL']
data_true = data[  data['LABEL'] == '__label1__' ]
data_shortened = data_true[features]

data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('N', 0)
data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('Y', 1)

data_shortened['LABEL'] = data_shortened['LABEL'].replace('__label1__', 1)
data_shortened['LABEL'] = data_shortened['LABEL'].replace('__label2__', 0)

#data_pos = data_shortened[  data_shortened['RATING'] == 5 ]
#data_neg = data_shortened[data_shortened['RATING'] == 1]

#print(data_pos.head())

labels = data_shortened.pop('LABEL')

#print(data_pos.head())

#print(data_pos.dtypes)

<ipython-input-2-085cdb72aa0e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('N', 0)
<ipython-input-2-085cdb72aa0e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_shortened['VERIFIED_PURCHASE'] = data_shortened['VERIFIED_PURCHASE'].replace('Y', 1)
<ipython-input-2-085cdb72aa0e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [3]:


tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
tokenized_df = tokenizer(data_shortened.head()["REVIEW_TEXT"].tolist(), padding="max_length", truncation=True)

print(type(tokenized_df))
print(tokenized_df.keys())

<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [5]:

train_features = {x: tf.convert_to_tensor(tokenized_df[x]) for x in tokenizer.model_input_names}
#print(train_features)

#The dataset now is a tuple of dictionary containint inpud_ids, token_type_ids. attention masks        and  a tf tensor containing the value for the label
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, labels.head()))

#print(train_tf_dataset)
train_tf_dataset = train_tf_dataset.shuffle(5).batch(5)

print(train_tf_dataset)



<BatchDataset shapes: ({input_ids: (None, 512), token_type_ids: (None, 512), attention_mask: (None, 512)}, (None,)), types: ({input_ids: tf.int32, token_type_ids: tf.int32, attention_mask: tf.int32}, tf.int64)>


In [6]:
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=train_tf_dataset, epochs=3, verbose=1)

model.save_pretrained("transformer_model")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (o

In [10]:

#print(data_pos.head()["REVIEW_TEXT"].tolist()[0])

print(labels.head())

for i in range(5):
    y = tokenizer(data_shortened.head()["REVIEW_TEXT"].tolist()[i], padding="max_length", truncation=True)

    y['input_ids'] = tf.convert_to_tensor( [y['input_ids']] )
    y['token_type_ids'] = tf.convert_to_tensor( [y['token_type_ids']] )
    y['attention_mask'] = tf.convert_to_tensor( [y['attention_mask']] )

    label = model(y)
    probs = tf.nn.softmax(label.logits)
    print(probs)



0    1
1    1
2    1
3    1
4    1
Name: LABEL, dtype: int64
tf.Tensor([[0.2620531  0.73794687]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.22877496 0.77122504]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.23225027 0.76774967]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.2602506 0.7397495]], shape=(1, 2), dtype=float32)
tf.Tensor([[0.22276734 0.77723265]], shape=(1, 2), dtype=float32)
